## Filters

过滤器是资产和时刻的函数，它是一个布尔值：

![过滤器函数](./filter_fun.png)

在管道中，`Filters`用于缩小包含在计算中或流水线最终输出中的证券集合。有两种常用的方法来创建过滤器：`Factor/Classifier`方法。

## Comparison Operators

因子和分类器的比较运算产生过滤器。由于我们还没有看过`Classifiers`，所以仍然使用因子做例子。以下会生成一个筛选器，每当最新收盘价格高于20，就会返回`True`。

以下产生一个过滤器，当10天平均值低于30天平均值时返回`True`。

请记住，过滤器每天会为每个证券生成一个`True`或`False`值。

## Factor/Classifier Methods

因子和分类器类的各种方法返回过滤器。再次，因为我们还没有看过分类器，现在让我们继续使用因子方法（稍后我们将看看分类器方法）。

Factor.top(n)方法会生成一个过滤器，该过滤器每天对给定因子的前n个证券返回True。以下示例会生成一个筛选器，该筛选器每天返回前200个证券，表明这些证券在所有已知证券的最后收盘价中，排位于前200。

+ 有关返回过滤器的因子方法的完整列表，请参阅此[链接](https://www.quantopian.com/help#quantopian_pipeline_factors_Factor)。

+ 有关返回过滤器的分类器方法的完整列表，请参阅此[链接](https://www.quantopian.com/help#quantopian_pipeline_classifiers_Classifier)。

## Dollar Volume Filter
以成交额自定义因子方法生成过滤器

让我们创建一个过滤器，如果证券的30天平均美元交易量高于10,000,000美元，则返回True。为此，我们首先需要创建`AverageDollarVolume`因子来计算30天的平均美元交易量。

要导入内置的AverageDollarVolume过滤器，我们可以添加到我们用于导入`SimpleMovingAverage`的行。

In [1]:
from zipline.pipeline.factors import AverageDollarVolume, SimpleMovingAverage
from zipline.pipeline import Pipeline

然后我们可以实例化因子。

In [2]:
from zipline.pipeline.data import CNEquityPricing

In [3]:
dollar_volume = AverageDollarVolume(window_length=30, inputs=[CNEquityPricing.close, CNEquityPricing.volume])

默认情况下，`AverageDollarVolume`使用`USEquityPricing.close`和`USEquityPricing.volume`作为输入，所以我们不需要指定它们。

底层代码已经将其更改为`CNEquityPricing`。

现在我们有一个美元成交量因子，我们可以用布尔表达式创建一个过滤器。以下行创建一个过滤器，其返回True的值为dollar_volume大于`LIMIT`的证券：

In [4]:
LIMIT = 10000*10000

要查看此过滤器的输出，我们可以将其作为列添加到我们的管道中。

In [5]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(
        inputs=[CNEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(
        inputs=[CNEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(window_length=3)

    high_dollar_volume = (dollar_volume > LIMIT)

    return Pipeline(
        columns={
            'percent_difference': percent_difference,
            'high_dollar_volume': high_dollar_volume,
            'dollar_volume':dollar_volume,
            'volume':CNEquityPricing.volume.latest,
        }, )

如果我们创建并运行我们的管道，现在我们有一个列`high_dollar_volume`，其布尔值对应于每个证券的表达式结果。

In [6]:
from zipline.research import run_pipeline

In [7]:
result = run_pipeline(make_pipeline(), '2018-12-27', '2018-12-27')
result.head(30)

percent_difference  \
datetime                  asset                                
2018-12-27 00:00:00+00:00 平安银行(000001)             -0.039134   
                          万 科Ａ(000002)             -0.004854   
                          国农科技(000004)              0.000596   
                          世纪星源(000005)             -0.034224   
                          深振业Ａ(000006)             -0.007235   
                          全新好(000007)               0.030670   
                          神州高铁(000008)             -0.011661   
                          中国宝安(000009)             -0.049069   
                          美丽生态(000010)             -0.121482   
                          深物业A(000011)             -0.006827   
                          南 玻Ａ(000012)             -0.028013   
                          沙河股份(000014)              0.008552   
                          深康佳Ａ(000016)             -0.062261   
                          *ST中华A(000017)           -0.011672   
                          深粮控股(000019)             -0.025162   
                          深华发Ａ(000020)              0.031705   
                          深科技(000021)              -0.023401   
                          深天地Ａ(000023)              0.061139   
                          特 力Ａ(000025)             -0.021515   
                          飞亚达(000026)              -0.013303   
                          深圳能源(000027)             -0.012973   
                          国药一致(000028)             -0.041658   
                          富奥股份(000030)             -0.024886   
                          大悦城(000031)              -0.017059   
                          深桑达Ａ(000032)             -0.022702   
                          神州数码(000034)             -0.052919   
                          中国天楹(000035)              0.069329   
                          华联控股(000036)             -0.011586   
                          深南电A(000037)             -0.024652   
                          深大通(000038)              -0.021853   

                                          high_dollar_volume  dollar_volume  \
datetime                  asset                                               
2018-12-27 00:00:00+00:00 平安银行(000001)                  True   4.730489e+08   
                          万 科Ａ(000002)                  True   9.098059e+08   
                          国农科技(000004)                 False   3.390361e+06   
                          世纪星源(000005)                 False   9.680041e+06   
                          深振业Ａ(000006)                 False   5.291878e+07   
                          全新好(000007)                   True   2.060412e+08   
                          神州高铁(000008)                  True   1.060769e+08   
                          中国宝安(000009)                 False   5.397226e+07   
                          美丽生态(000010)                 False   1.743081e+07   
                          深物业A(000011)                 False   3.532370e+07   
                          南 玻Ａ(000012)                 False   1.998462e+07   
                          沙河股份(000014)                 False   3.645141e+07   
                          深康佳Ａ(000016)                 False   3.278389e+07   
                          *ST中华A(000017)               False   3.919715e+07   
                          深粮控股(000019)                 False   2.760702e+07   
                          深华发Ａ(000020)                 False   1.888417e+07   
                          深科技(000021)                  False   3.164372e+07   
                          深天地Ａ(000023)                 False   6.022210e+07   
                          特 力Ａ(000025)                 False   5.463862e+07   
                          飞亚达(000026)                  False   1.114995e+07   
                          深圳能源(000027)                 False   1.473458e+07   
                          国药一致(000028)                 False   1.963142e+07   
                          富奥股份(000030)                 False   8.928374e+06   
                  

## Applying a Screen

默认情况下，管道每天为Quantopian数据库中的每个资产生成计算值。 然而，很多时候，我们只关心符合特定标准的证券子集（例如，我们可能只关心有足够的每日交易量来快速填补我们订单的证券）。我们可以通过将筛选器通过`screen`关键字传递给我们的管道来告诉我们的管道忽略过滤器产生`False`的证券。

为了筛选我们的30天平均美元数量大于10,000,000美元证券的管道输出，我们可以简单地将我们的`high_dollar_volume`过滤器作为屏幕参数。 这就是我们的`make_pipeline`函数现在的样子：

In [8]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(
        inputs=[CNEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(
        inputs=[CNEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(
        window_length=30,
        inputs=[CNEquityPricing.close, CNEquityPricing.volume])
    high_dollar_volume = (dollar_volume > LIMIT)

    return Pipeline(
        columns={'percent_difference': percent_difference},
        screen=high_dollar_volume)

运行此操作将仅为在给定日期通过high_dollar_volume的证券生成输出。 例如，2018年4月10日运行这条管道可以产生约1100只股票。

In [9]:
result = run_pipeline(make_pipeline(), '2018-04-10', '2018-04-10')
print('Number of securities that passed the filter: %d' % len(result))

Number of securities that passed the filter: 1110


## Inverting a Filter

`~`运算符用于反转过滤器，将所有`True`值与`False`交换，反之亦然。例如，我们可以编写以下内容来筛选低美元数量的证券：

low_dollar_volume = ~high_dollar_volume

对于在过去30天内平均美元交易量低于或等于`LIMIT`的所有证券，这将返回`True`。

在下一课中，我们将介绍如何组合滤镜。